In [10]:
import pandas as pd
import os

In [9]:
covid_ds = "../Dataset/covid/"
politics_ds = "../Dataset/politics/"

In [4]:
covid_anno = pd.read_json(f"{covid_ds}/annotations/train.jsonl", lines = True).reset_index(drop=True)
politics_anno = pd.read_json(f"{politics_ds}/annotations/train.jsonl", lines = True).reset_index(drop=True)

In [5]:
covid_anno[covid_anno.image == "covid_memes_178.png"]

,OCR,image,hero,villain,victim,other
130,When you're on a crowded MRT and\nsomeone star...,covid_memes_178.png,[],[],[],[mass rapid transit (mrt)]


In [6]:
covid_entities = covid_anno.apply(lambda x: x.hero + x.villain + x.victim + x.other, axis=1)
politics_entities = politics_anno.apply(lambda x: x.hero + x.villain + x.victim + x.other, axis=1)

In [1]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

print(fuzz.ratio("Bernie Sanders","Bernie Sanders")
    ,fuzz.partial_ratio("Bernie Sanders","Bernie")
    ,fuzz.token_sort_ratio("Bernie Sanders","Bernie")
    )

100 100 60


In [7]:
def get_fuzz_match(value, ocr, threhsold):
    if value in ocr:
        return True
    else:
        tokens = ocr.replace("\n","").split(" ")
        for _token in tokens:
            r = fuzz.ratio(value,_token)
            pr = fuzz.partial_ratio(value,_token)
            sr = fuzz.token_sort_ratio(value,_token)

            if (r>threhsold) or (pr > threhsold) or (sr>threhsold):
                return True
        
        return False

In [8]:
image_entities = {}
def get_image_entities():
    
    _iterrables = [(covid_entities, covid_anno), (politics_entities, politics_anno)]
    for _iterable,df in _iterrables:
        for index, _list in _iterable.iteritems():
            count = 0
            _imgentities = []
            ocr = df.loc[index,"OCR"]
            image = df.loc[index,"image"]
            for _item in _list:
                if not get_fuzz_match(_item.lower(), ocr.lower(),70):
                    count+=1
                    _imgentities.append(_item)
                    if _item in image_entities:
                        image_entities[_item]["Count"] +=1
                        image_entities[_item]["image"].append(image)
                    else:
                        image_entities[_item] = {}
                        image_entities[_item]["Count"] = 1
                        image_entities[_item]["image"] = [image]

            df.loc[index,"TotalObjectsInImage"] = count
            df.loc[index, "ImageEntity"] = ",".join(_imgentities)

get_image_entities()

In [94]:
covid_anno.TotalObjectsInImage.value_counts()

0.0    2079
1.0     444
2.0     126
3.0      42
4.0       9
Name: TotalObjectsInImage, dtype: int64

## Analyze political dataset

In [95]:
politics_anno.TotalObjectsInImage.value_counts()

0.0    1627
1.0     799
2.0     295
3.0      94
4.0      23
5.0      10
6.0       2
7.0       2
Name: TotalObjectsInImage, dtype: int64

In [96]:
politics_anno.sort_values(by = "TotalObjectsInImage", ascending=False)

,OCR,image,hero,villain,victim,other,TotalObjectsInImage,ImageEntity
2718,Which of these\ntwo corrupt & stupid\ngroups i...,memes_1449.png,[],"[mainstream media, dem socialists]",[],"[political party, corporates, cnn, nbc, social...",7.0,"mainstream media,dem socialists,political part..."
2214,WHAT THE HELL HAPPENED?\n,memes_5893.png,[],[donald trump],[],"[abraham lincoln, ted cruz, john f. kennedy, n...",7.0,"donald trump,abraham lincoln,ted cruz,john f. ..."
910,The Other\nGREEN Party\n#JilINotHill\n,memes_2071.png,[],"[donald trump, hiliary clinton]",[],"[green party, 2016, jill stein, grren party, h...",6.0,"donald trump,hiliary clinton,2016,jill stein,g..."
743,REMEMBER WHEN POLITICS ATTRACTED\nTHE BRIGHTES...,memes_322.png,"[john f. kennedy, abraham lincoln, george wash...","[donald trump, sarah palin, ted cruz]",[],[politics],6.0,"john f. kennedy,abraham lincoln,george washing..."
1125,wers\nEveryone is worried\nabout the clowns on...,memes_6237.png,[],"[donald trump, hiliary clinton]",[],"[street, carly fiorina, president, clowns, joe...",5.0,"donald trump,hiliary clinton,carly fiorina,joe..."
...,...,...,...,...,...,...,...,...
1381,A CHEVY CORVAIR CLUB\nWHEN THEY HEAR RALPH NAD...,memes_1434.png,[],[],[],"[corvair club, ralph nader]",0.0,
1379,NO SURPRISE THE DEMOCRATS\nOFFICIALLY VOTED TO...,memes_1295.png,[],[democrats],[],"[democratic party, halloween, hillary clinton,...",0.0,
1378,MAKS AMERICA\nGSAT AGAIN\nHow sad it must be-\...,memes_5908.png,[],"[historians, scientists, economists, scholars,...",[],"[reality tv stars, reality tv star]",0.0,
1377,POINTING FINGERS\nPoty ma ol te noden Reptkon ...,memes_4699.png,[],[],[],"[cheezeburger, memes]",0.0,


In [141]:
politics_anno[(politics_anno.TotalObjectsInImage == 1) & (politics_anno.ImageEntity == "david cameron")]

,OCR,image,hero,villain,victim,other,TotalObjectsInImage,ImageEntity
2705,IF YOU THINK EVERYTHING WILL\nBE OK ONCE WE GE...,memes_2219.png,[],[david cameron],[],[],1.0,david cameron


In [124]:
po1_set = set()
po_1 = politics_anno[politics_anno.TotalObjectsInImage == 1].ImageEntity.value_counts()
po_1.to_csv("../Datasets/politics_1.csv")
for _item in po_1.index.values:
    po1_set.add(_item)

In [127]:
# Number of images more than 1 entities that are also present in images with only 1 entity
po2_set = set()
po_2 = politics_anno[politics_anno.TotalObjectsInImage == 2].ImageEntity.values
for _item in po_2:
    po2_set.update(set(_item.split(",")))


In [130]:
intersect = po2_set.intersection(po1_set)
len(intersect),len(po2_set), len(po1_set)

(50, 151, 150)

In [136]:
po2_po1_intersection = set()
threhsold = 80
for _oitem in po2_set:
    for _iitem in po1_set:
            r = fuzz.ratio(_oitem,_iitem)
            pr = fuzz.partial_ratio(_oitem,_iitem)
            sr = fuzz.token_sort_ratio(_oitem,_iitem)

            if (r>threhsold) or (pr > threhsold) or (sr>threhsold):
                po2_po1_intersection.add((_oitem, _iitem))
        

In [137]:
po2_po1_intersection

{('2020 election', 'election'),
 ('?', '?'),
 ('abc news', 'abc news'),
 ('abc news', 'msnbc news'),
 ('abraham lincoln', 'abraham lincoln'),
 ('abraham lincoln', 'abrahm lincoln'),
 ('abrahm lincoln', 'abraham lincoln'),
 ('abrahm lincoln', 'abrahm lincoln'),
 ('adolf hilter', 'adolf hitler'),
 ('adolf hitler', 'adolf hitler'),
 ('america', 'america'),
 ('america', 'american flag'),
 ('america', 'americans'),
 ('america', 'captain america civil war'),
 ('barack obama', 'barack obama'),
 ('barack obama', 'barack obama memes'),
 ('barack obama memes', 'barack obama'),
 ('barack obama memes', 'barack obama memes'),
 ('barack obama memes', 'meme'),
 ('barck obama', 'barack obama'),
 ('barck obama', 'barack obama memes'),
 ('bernie sanders', 'bernie sanders'),
 ('bill clinton', 'bill clinton'),
 ('bill clinton', 'hillary clinton'),
 ('black people', 'black people'),
 ('black people', 'people'),
 ('car', 'ben carson'),
 ('car', 'carey abe'),
 ('chris wallace', 'chris wallace'),
 ('chuck sch

## Covid Analysis

In [142]:
covid_anno.sort_values(by = "TotalObjectsInImage", ascending=False)

,OCR,image,hero,villain,victim,other,TotalObjectsInImage,ImageEntity
1999,pert Say Deay wan Can edy\nAkahal&Hgh Terpert\...,covid_memes_4926.png,[],[],[],"[wuhan virus, china, bars, coronavirus, alcoho...",4.0,"wuhan virus,coronavirus,alcohol,corona beer"
2285,Kat\natoeswenth\nNcoronavirusmemes\nCurrency i...,covid_memes_4193.png,"[purell, vodka, currency, lysol, toilet paper,...",[],[],"[sanitizer, disinfectant spray, coronavirus]",4.0,"vodka,lysol,toilet paper,sanitizer"
2267,"NO00, DO NOT\nWFH BE LIKE\n",covid_memes_4130.png,[],[],[],"[toilet paper, work from home (wfh), work from...",4.0,"toilet paper,work from home (wfh),work from ho..."
810,ACHOO!\nEXPECTO CORONAS!\n,covid_memes_1483.png,[],[],[],"[hermione, daniel radcliffe, coronavirus, harr...",4.0,"hermione,daniel radcliffe,harry potter,emma ch..."
2155,Social Greetings Ranked by Chance\nof Infectio...,covid_memes_3832.png,"[namaste, vulcan]","[high five, handshake, kiss, fist bump]",[],[social greetings],4.0,"namaste,high five,kiss,fist bump"
...,...,...,...,...,...,...,...,...
1253,hengyytoy.en.alibaba.com\nThe Frog Pepe Latex ...,covid_memes_2349.png,[],[],[],"[mask, donald trump]",0.0,
1252,Chance\na virus with\na 99.97%\nrecovery rate\...,covid_memes_5244.png,[],[],[],"[vaccine, coronavirus, virus, experimental vac...",0.0,
1251,If you are unnecessarily roaming outside\nwe w...,covid_memes_2030.png,[],[],[],[masakkali],0.0,
325,Wuhan virus\nYou're racist\nThen why do you us...,covid_memes_227.png,[],[],[],"[wuhan virus, spanish flu, wuhan coronavirus, ...",0.0,


In [144]:
co1_set = set()
co1 = covid_anno[covid_anno.TotalObjectsInImage == 1].ImageEntity.value_counts()
co1.to_csv("../Datasets/covid_1.csv")
for _item in co1.index.values:
    co1_set.add(_item)

In [117]:
def search_term(term):
    all_searches = []
    _iterrables = [(covid_entities, covid_anno), (politics_entities, politics_anno)]
    for _iterable,df in _iterrables:
        for index, _list in _iterable.iteritems():
            if term in _list:
                all_searches.append(df.loc[index,["OCR","image"]])

    return all_searches

In [14]:
covid_anno[(covid_anno.TotalObjectsInImage == 1) & (covid_anno.ImageEntity == "coronavirus")].sort_values(by = "image").values[:1]

array([['butterchickendedo\n@ymunjal0001\nPeople making momos, dalgona\ncoffee, burgers etc.\nCorona:\nYaha toh pary ho rhi hai\nbutterchickendedo\n',
        'covid_memes_1053.png', list([]), list([]), list([]),
        list(['coronavirus', 'dalgona coffee', 'burgers', 'momos']), 1.0,
        'coronavirus']], dtype=object)

### Analyze interested entities

In [17]:
concat_ds

,OCR,image,hero,villain,victim,other,TotalObjectsInImage,ImageEntity
0,Trump: Does he always just stand there\nlike t...,memes_2801.png,[],[],[],"[donald trump, joe biden, barack obama]",2.0,"joe biden,barack obama"
1,WELCOME TO THE REPUBLICAN\nPARTY\nWHERE THE PO...,memes_4527.png,[],[republican party],"[women, poor, minorities]",[],0.0,
2,"NOW THAT RUSSIA COLLUSION IS\nPROVEN LIE, WHEN...",memes_862.png,[],"[bill clinton, barack obama, hillary clinton]",[],"[russian, russia]",3.0,"bill clinton,barack obama,hillary clinton"
3,"Biden:I changed the\nWiFi password to\n""ILoveM...",memes_2884.png,[],[donald trump],[mexicans],"[wifi, joe biden, barack obama]",1.0,barack obama
4,"""The power to create\nthis new world\nis not i...",memes_2208.png,[],[],[],"[jil stein, jill stein]",0.0,
...,...,...,...,...,...,...,...,...
5547,GO CORONA\nCORONA GO\nBuddyBits\n,covid_memes_5288.png,[],[],[],"[coronavirus, corona]",1.0,coronavirus
5548,betef eh erpn\nMY PLANS\n2020\n,covid_memes_5297.png,[],[2020],[my plans],[plans],0.0,
5549,INDIA\nITALY\nwe have over 10k\ninfected with\...,covid_memes_5303.png,[],[china],"[italy, india, usa]","[coronavirus, usa. china]",0.0,
5550,"Coronavirus: ""intects Tom Hanks""\nowator Folov...",covid_memes_5321.png,[],[],[tom hanks],"[keanu reeves, coronavirus]",1.0,keanu reeves


In [27]:
def is_matching_pair(value1, value2, threhsold):
    r = fuzz.ratio(value1,value2)
    pr = fuzz.partial_ratio(value1,value2)
    sr = fuzz.token_sort_ratio(value1,value2)

    if (r>threhsold) or (pr > threhsold) or (sr>threhsold):
        return True
    else:
        return False
        
concat_ds = pd.concat([politics_anno, covid_anno], axis = 0, ignore_index=True).reset_index(drop = True)
concat_ds
master_entities = pd.read_csv("../Dataset/MasterEntities.csv")
_filtered_ds = concat_ds.loc[(concat_ds.TotalObjectsInImage == 1),["ImageEntity","image"]]

matching_pairs = []
master_entities
for item in master_entities.values:
    _entity = item[0]
    for inner_index, inner_item in _filtered_ds.iterrows():
        is_matched = is_matching_pair(_entity, inner_item.ImageEntity, 70)
        if is_matched:
            matching_pairs.append((inner_index, inner_item.image, _entity, inner_item.ImageEntity))

In [29]:
x = pd.DataFrame(matching_pairs, columns = ["Index", "Image","OriginalEntity","PartialMatch"])
x.to_csv("../Dataset/matched_pairs.csv", index=False)

In [16]:
matchedpairs = pd.read_csv("../Dataset/matched_pairs.csv")
agg = matchedpairs.groupby("OriginalEntity")["Image"].count().reset_index().sort_values(by="Image", ascending=False)
print(agg.shape)
#agg

(24, 2)


In [11]:
import shutil

def copyimage(imagename,index):
    if filename.startswith("covid"):
        shutil.copyfile(f"{covid_ds}/images/{filename}", f"../Dataset/images/{filename-str(index)}")
    else:
        shutil.copyfile(f"{politics_ds}/images/{filename}", f"../Dataset/images/{filename-str(index)}")

In [46]:
# Create a separate subset of images for image classification purpose

'''
matchedpairs = pd.read_csv("../Datasets/matched_pairs.csv")
agg = matchedpairs.groupby("OriginalEntity")["Image"].count().reset_index().sort_values(by="Image", ascending=False)
print(agg.shape)

for _, item in matchedpairs.iterrows():
    filename = item.Image
    copyimage(filename, 0)
'''

(797, 4)


In [18]:
agg[agg.Image <=10]

,OriginalEntity,Image
21,ted cruz,10
13,kamala harris,8
17,mitch mcconnell,7
23,xi jinping,6
16,mike pence,6
22,vladimir putin,6
7,gary johnson,6
10,jill stein,6
12,john f. kennedy,5
4,bill clinton,5


In [20]:
# Replicate images whihc are present less than 10


for _, item in agg[agg.Image <=10].iterrows():
    _entityname = item.OriginalEntity
    _filteredmatch = matchedpairs[matchedpairs.OriginalEntity == _entityname]
    for _index in range(0,4):
        matchedpairs = pd.concat([matchedpairs,_filteredmatch ], axis = 0, ignore_index=True)

In [22]:
matchedpairs.shape
#matchedpairs.groupby("OriginalEntity")["Image"].count().reset_index().sort_values(by="Image", ascending=False)
matchedpairs.to_csv("../Dataset/matchedpairs_finalds.csv", index = False)